In [ ]:
import numpy as np
import serial
from queue import Queue
import time
import sys
from IPython.display import clear_output

In [ ]:
COM = '/dev/cu.wchusbserial1420'# /dev/ttyACM0 (Linux)
BAUD = 115200
LENGTH = 50

class SensorDaemon:
    def __init__(self, length = 10, is_notebook=True, print_values=False):
        self.ser = serial.Serial(COM, BAUD, timeout = .1)
        self.keys = ['G', 'M', "A"]
        self.length = length
        self.g_display = []
        self.m_display = []
        self.a_display = []
        self.g = np.empty([1,3])
        self.m = np.empty([1,3])
        self.a = np.empty([1,3])
        self.vibration = 0
        self.on = False
        self.light = 0
        self.start_time = None
        self.end_time = None
        self.is_notebook = is_notebook
        self.print_values = print_values

    def run(self, seconds = 60):
        try:
            while True:
                out = str(self.ser.readline().decode())
                if out and out[0] in self.keys:
                    key = out[0]
                    try:
                        vals =[int(_) for _ in out[:-2].replace(" ","").split(',')[1:]]
                        self.add_vals(key,vals)
                        if not self.start_time:
                            self.start_time = time.time()
                        else:
                            t = time.time()
                            if t >= (self.start_time+seconds):
                                self.stop()
                                break
                    except ValueError:
                        pass
                    finally:
                        if self.display:
                            self.display()
        except KeyboardInterrupt:
            self.stop()
            
    def stop(self):
        self.end_time = t
        self.ser.write("v \n".encode())
        self.ser.write("r \n".encode())
        print("\nDaemon stopped")
            
    def write(self, string):
        self.ser.write((string+"\n").encode())
        
        
    def add_vals(self, key, value):
        if key == "G":
            self.g = np.vstack([self.g, value])
            self.g_display = value
        elif key == "M":
            self.m = np.vstack([self.m, value])
            self.m_display = value
        elif key == "A":
            self.a = np.vstack([self.a, value])
            self.a_display = value
    
    def display(self):
        g = self.g_display
        m = self.m_display
        a = self.a_display
        t = time.time()-self.start_time
        if self.is_notebook:
            clear_output(wait=True)
            sys.stdout.write('G: {} \nM: {}\nA: {} \ntime: {}'.format(g,m,a,t))
            sys.stdout.flush()
        else:
            print('\r G: {} \nM: {}\nA: {} \ntime: {}'.format(g,m,a,t), end="")
    
    def save_csv(self, filename="output.csv"):
        timestamp = [self.start_time, self.end_time, self.end_time-self.start_time]
        self.g[0] = timestamp
        self.m[0] = timestamp
        self.a[0] = timestamp
        np.savetxt("g_"+filename, self.g, delimiter=',')
        np.savetxt("m_"+filename, self.m, delimiter=',')
        np.savetxt("a_"+filename, self.a, delimiter=',')
        print("Saved")

In [ ]:
d = SensorDaemon(print_values = True)
d.run(seconds = 5)

In [ ]:
print("Start time:", d.start_time)
print("End time:", d.end_time)
print("Elapsed time:", d.end_time - d.start_time)

In [ ]:
user = "114"
d.save_csv(filename="{}.csv".format(user))